# 如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程

此範例程式碼為文章「如何在 Colab 安裝 Darknet 框架訓練 YOLO v3 物件辨識並且最佳化 Colab 的訓練流程」的範例程式碼。

這篇文章會教你：
* 利用 Colab 128G RAM GPU 來訓練你的 Yolo3 模型
* 將 Colab 設定成可以運用在實際專案的訓練環境
* 快速掛載本機電腦上的檔案到 Colab 環境中
* 事先編譯所有需要的檔案，每次開啟 Colab 後即可立刻進行訓練
* 將訓練好的 weight 檔案自動同步回本機電腦，避免檔案遺失

## Step 2：連接你的 Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') # 請點擊網址，選擇 Google 帳號登入，然後將授權碼貼回輸入框中

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!ln -fs /content/gdrive/My\ Drive /app

ln: failed to create symbolic link '/app/My Drive': Input/output error


## 步驟 4：下載並編譯 Darknet（只需要執行一次）

這個步驟只需要執行一次，執行過程我們會把執行檔複製到 Google Drive。之後要跑程式時，只要把執行檔複製回 Colab 環境就可以使用了。

### 4.1 下載 cuDNN 檔案

1. 申請 Nvidia 帳號，申請網址為 http://bit.ly/2qfpOPj
2. 下載 `cudnn-10.0-linux-x64-v7.5.0.56.tgz`，下載網址為
下載 cuDNN 檔案。下載網址為：http://bit.ly/2qfpOPj
3. 將下載的檔案 `cudnn-10.0-linux-x64-v7.5.0.56.tgz` 放到 google drive 的 `darknet/cuDNN/` 目錄下。

In [ ]:
!tar -Jxvf /app/cudnn/cudnn-linux-x86_64-8.9.1.23_cuda11-archive.tar.xz -C /usr/local/
#!dpkg -i /app/cudnn/cudnn-local-repo-ubuntu2004-8.9.1.23_1.0-1_amd64.deb
!chmod a+r /usr/local/cuda/include/cudnn.h

# 檢查是否安裝成功
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2

cudnn-linux-x86_64-8.9.1.23_cuda11-archive/
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_adv_infer_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_adv_infer_static_v8.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_adv_train_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_adv_train_static_v8.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_cnn_infer_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_cnn_infer_static_v8.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_cnn_train_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_cnn_train_static_v8.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_ops_infer_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_ops_infer_static_v8.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_ops_train_static.a
cudnn-linux-x86_64-8.9.1.23_cuda11-archive/lib/libcudnn_ops_train_st

In [ ]:
#!find / -name cudnn.h -print
#!cp /usr/include/cudnn_version.h /content
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 8
#define CUDNN_MINOR 7
#define CUDNN_PATCHLEVEL 0
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


### 4.2 下載並編譯 Darknet

In [ ]:
!apt install libopencv-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libopencv-dev is already the newest version (4.2.0+dfsg-5+focal2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
!apt-get install g++
#!pkg-config --modversion opencv
#!pkg-config --cflags opencv
#!find / -name opencv.pc -print

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:9.3.0-1ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [ ]:
%cd /content/
!git clone https://github.com/kriyeng/darknet/ darknet_source
%cd darknet_source
!cp /app/cudnn/Makefile .
# 修改 Darknet 設定，符合 Colab 環境
!sed -i "s/GPU=0/GPU=1/g" Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" Makefile

# 編譯
!make

# 將編譯完成的執行檔複製回 Google Drive
!cp ./darknet /app/darknet
!cp ./darknet /content/darknet
%cd /content

/content
fatal: destination path 'darknet_source' already exists and is not an empty directory.
/content/darknet_source
chmod +x *.sh
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(mat_cv*, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:896:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  896 |                 float rgb[3];
      |                       ^~~
g++ -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_

# 測試 Darknet 是否正常安裝

我們用 yolo 官方預先訓練好的模型來偵測範例圖片。

In [ ]:
%cd /content/darknet_source
!wget https://pjreddie.com/media/files/yolov3.weights
!/content/darknet detect cfg/yolov3.cfg yolov3.weights data/person.jpg -dont-show



/content/darknet_source
--2023-05-11 13:54:00--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  20.9MB/s    in 12s     

2023-05-11 13:54:13 (19.6 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]

/bin/bash: /content/darknet: No such file or directory


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
img = cv2.imread('predictions.jpg', cv2.IMREAD_UNCHANGED)
cv2_imshow(img)

AttributeError: ignored